In [1]:
!pip install contractions
!pip install langdetect

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 6.2 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 12.7 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993225 sha256=a6abc1c00fbdf4e328ea916ee67427258efc63d9f0caa8d285814e4fbd79c12c
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [2]:
import os
import pandas as pd
import re
import nltk
import contractions
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')  # Downloading the required wordnet data
nltk.download('punkt')
from langdetect import detect

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /usr/share/nltk_data...
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
    stop_words = set(stopwords.words('english'))
    return ' '.join([word for word in text.split() if word not in stop_words])
def tokenize_text(text):
    return word_tokenize(text)

In [4]:
def preprocess(df):
    # Convert 'combined' column to strings
    df['combined'] = df['combined'].astype(str)
    
    # Lowercase
    df['combined'] = df['combined'].str.lower()

    # Expanding Contractions
    df['combined'] = df['combined'].apply(lambda x: contractions.fix(x))

    # Removing the URLs
    df['combined'] = df['combined'].apply(lambda x: re.sub(r'http\S+|www\S+|https\S+', '', x, flags=re.MULTILINE))

    # Removing the HTML tags
    df['combined'] = df['combined'].apply(lambda x: re.sub(r'<.*?>', '', x))

    # Removing the numbers
    df['combined'] = df['combined'].apply(lambda x: re.sub(r'\d+', '', x))

    # Removing the extra whitespaces
    df['combined'] = df['combined'].apply(lambda x: x.strip())

    # Removing the punctuation
    df['combined'] = df['combined'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

    # Removing the stopwords
    df['combined'] = df['combined'].apply(remove_stopwords)

    # Tokenization
    df['combined'] = df['combined'].apply(tokenize_text)

    return df

# XgBoost

In [5]:
!pip install xgboost

In [6]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import xgboost as xgb
import ast
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report


In [7]:
train = pd.read_csv('/kaggle/input/dataset-cleaned/UTK/clean_train_UTK.csv')

In [8]:
train = preprocess(train)

In [9]:
train.head()

,id,title,text,label,combined
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,House Dem Aide: We Didn’t Even See Comey’s Let...,1,"[house, dem, aide, even, see, comeys, letter, ..."
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Ever get the feeling your life circles the rou...,0,"[flynn, hillary, clinton, big, woman, campus, ..."
2,2,Why the Truth Might Get You Fired,"Why the Truth Might Get You Fired October 29, ...",1,"[truth, might, get, fired, truth, might, get, ..."
3,3,15 Civilians Killed In Single US Airstrike Hav...,Videos 15 Civilians Killed In Single US Airstr...,1,"[civilian, killed, single, airstrike, identifi..."
4,4,Iranian woman jailed for fictional unpublished...,Print \r\nAn Iranian woman has been sentenced ...,1,"[iranian, woman, jailed, fictional, unpublishe..."


In [10]:
X_train = train['combined']
y_train = train['label']

In [11]:
X_train.head()

0    [house, dem, aide, even, see, comeys, letter, ...
1    [flynn, hillary, clinton, big, woman, campus, ...
2    [truth, might, get, fired, truth, might, get, ...
3    [civilian, killed, single, airstrike, identifi...
4    [iranian, woman, jailed, fictional, unpublishe...
Name: combined, dtype: object

In [12]:
X_train = X_train.apply(' '.join)

In [16]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.datasets import make_classification

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),
    ('clf', LogisticRegression(max_iter=1000, solver='liblinear')),  # Logistic Regression classifier
])

# Define the grid parameters
params = {
    'clf__C': [0.1, 1, 10, 100],  # Regularization parameter
    'clf__penalty': ['l1', 'l2'],  # Regularization type
    'clf__solver': ['liblinear', 'saga'],  # Solver type
}

# Create a StratifiedKFold object
stratified_kfold = StratifiedKFold(n_splits=5)

# Create the grid search object
grid_search = GridSearchCV(pipeline, params, cv=stratified_kfold, n_jobs=-1, refit=True, verbose=2)

# Fit the grid search object to the data to compute the optimal model
grid_search.fit(X_train, y_train)

# Print the best score and the best parameters
print("Best Score: ", grid_search.best_score_)
print("Best Parameters: ", grid_search.best_params_)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which 

[CV] END .clf__C=0.1, clf__penalty=l1, clf__solver=liblinear; total time=  15.2s
[CV] END ......clf__C=0.1, clf__penalty=l1, clf__solver=saga; total time=  17.7s
[CV] END .clf__C=0.1, clf__penalty=l2, clf__solver=liblinear; total time=  14.4s
[CV] END .clf__C=0.1, clf__penalty=l2, clf__solver=liblinear; total time=  14.8s
[CV] END ......clf__C=0.1, clf__penalty=l2, clf__solver=saga; total time=  14.9s
[CV] END ...clf__C=1, clf__penalty=l1, clf__solver=liblinear; total time=  14.5s
[CV] END ...clf__C=1, clf__penalty=l1, clf__solver=liblinear; total time=  14.4s
[CV] END ........clf__C=1, clf__penalty=l1, clf__solver=saga; total time=  39.1s
[CV] END ...clf__C=1, clf__penalty=l2, clf__solver=liblinear; total time=  15.7s
[CV] END ...clf__C=1, clf__penalty=l2, clf__solver=liblinear; total time=  14.8s
[CV] END ...clf__C=1, clf__penalty=l2, clf__solver=liblinear; total time=  15.9s
[CV] END ........clf__C=1, clf__penalty=l2, clf__solver=saga; total time=  15.7s
[CV] END ..clf__C=10, clf__p

/opt/conda/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [17]:
clf = grid_search.best_estimator_

In [18]:
# Training performance
y_train_pred = clf.predict(X_train)
print(classification_report(y_train, y_train_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     10211
           1       1.00      1.00      1.00      8367

    accuracy                           1.00     18578
   macro avg       1.00      1.00      1.00     18578
weighted avg       1.00      1.00      1.00     18578

[CV] END .clf__C=0.1, clf__penalty=l1, clf__solver=liblinear; total time=  15.0s
[CV] END ......clf__C=0.1, clf__penalty=l1, clf__solver=saga; total time=  17.9s
[CV] END ......clf__C=0.1, clf__penalty=l1, clf__solver=saga; total time=  18.2s
[CV] END ......clf__C=0.1, clf__penalty=l2, clf__solver=saga; total time=  15.9s
[CV] END ......clf__C=0.1, clf__penalty=l2, clf__solver=saga; total time=  15.8s
[CV] END ...clf__C=1, clf__penalty=l1, clf__solver=liblinear; total time=  14.6s
[CV] END ........clf__C=1, clf__penalty=l1, clf__solver=saga; total time=  38.2s
[CV] END ........clf__C=1, clf__penalty=l1, clf__solver=saga; total time= 1.4min
[CV] END ..clf__C=10, clf

# Submit test prediction

In [19]:
test = pd.read_csv('/kaggle/input/dataset-cleaned/test.csv')

In [20]:
# Combine `text` and `title` columns into one column
test['combined'] = test['title'] + ' ' + test['text']

In [21]:
test.fillna('', inplace=True)
test = preprocess(test)
X_test = test['combined']
X_test = X_test.apply(' '.join)

In [22]:
id_test = test['id']
y_test_pred = grid_search.predict(X_test)

In [23]:
# Create a DataFrame
submission_df = pd.DataFrame({'id': id_test, 'label': y_test_pred})
# Save the DataFrame to a CSV file
submission_df.to_csv('submission1.csv', index=False)


In [24]:
# Create a DataFrame
submission_df = pd.DataFrame({'id': id_test, 'label': y_test_pred})
# Save the DataFrame to a CSV file
submission_df.to_csv('submission1.csv', index=False)

In [ ]:
from joblib import dump
dump(clf, 'lf_model.joblib')